In [ ]:
from fiat_toolbox.well_being import Household
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Prepare simple data for example
data = {
    'Household Type': ['Average', 'Poorest', 'Richest'],
    'Monthly Income': [4000, 2000, 9000],
    'Asset Value': [30000, 15000, 45000],
    'Relative Damage': [0.7, 0.7, 0.7],
    "plt_color": ["grey", "k", "purple"]
}
currency = "$"
df = pd.DataFrame(data)
df['Yearly Income'] = df['Monthly Income'] * 12
inc_avg = df.loc[df["Household Type"]=="Average", 'Yearly Income'].values[0]

df

In [ ]:
# Calculate asset loss
df['Asset Loss'] = df['Relative Damage'] * df['Asset Value']

# Melt the dataframe for seaborn
df_melted = df.melt(id_vars='Household Type', value_vars=['Asset Value', 'Asset Loss'], 
                    var_name='Type', value_name='Value')

# Plotting
plt.figure(figsize=(6, 4))
sns.barplot(x='Household Type', y='Value', hue='Type', data=df_melted)
plt.xlabel('Household Type')
plt.ylabel(currency)
plt.title('Asset Values and Asset Losses by Household Type')
plt.legend()
plt.show()

In [ ]:
#  Run the well-being model for each household type and optimize lambda
losses = {}
objects = {}

for i, household in df.iterrows():
    name = household['Household Type']
    var = Household(v=household['Relative Damage'], k_str=household['Asset Value'], c0=household['Yearly Income'], c_avg=inc_avg, currency=currency)
    var.opt_lambda()
    losses[name] = var.get_losses()
    objects[name] = var


losses = pd.DataFrame(losses)
losses

In [ ]:
# Check how the lambda optimization works for each household type
fig, axes = plt.subplots(2, 2, figsize=(10, 8), sharex='col', sharey='row')
ax1, ax2, ax3, ax4 = axes.flatten()

for i, row in df.iterrows():
    ts = objects[row["Household Type"]].l_opt
    ax1.plot(ts["lambda"], ts["Consumption Loss"], row["plt_color"], label=row["Household Type"])
    ax2.plot(ts["recovery_time"], ts["Consumption Loss"], row["plt_color"], label=row["Household Type"])
    ax3.plot(ts["lambda"], ts["Utility Loss"], row["plt_color"], label=row["Household Type"])
    ax4.plot(ts["recovery_time"], ts["Utility Loss"], row["plt_color"], label=row["Household Type"])
    
ax2.set_xscale('log')
ax4.set_xscale('log')
ax1.set_ylabel(f'Total Consumption Losses ({currency})')
ax3.set_ylabel('Total Wellbeing Losses')
ax3.set_xlabel('Reconstruction rate λ')
ax4.set_xlabel('Recovery Time (years)')
ax4.legend()

plt.tight_layout()

In [ ]:
# Plot lambda optimization for each household type
for i, row in df.iterrows():
    obj = objects[row["Household Type"]]
    print("λ optimization for ", row["Household Type"])
    obj.plot_opt_lambda()

In [ ]:
# Plot consumption over time for each household type
for i, row in df.iterrows():
    obj = objects[row["Household Type"]]
    print("Consumption over time for", row["Household Type"])
    obj.plot_consumption()

In [ ]:
# Select the specific rows for the plot
plot_data = losses.loc[['Asset Loss', 'Consumption Loss', 'Wellbeing Loss', 'Equity Weighted Loss']]

# Transpose the dataframe for plotting
plot_data = plot_data.T

# Plot the data
plot_data.plot(kind='bar')
plt.ylabel('Losses')
plt.xlabel('Household Type')
plt.title('Losses by Household Type')
plt.xticks(rotation=0)
plt.legend()
plt.show()